# Abstract
Что нужно сделать:
1. Скачать граф УДС и граф общественного транспорта
2. Создать модель (скачать трансформер (?) ), которая создает маршруты общественного транспорта
3. Создать модель, которая кластеризует (?) граф на сектора и добавляет экспресс-маршруты

## Import

In [22]:
import geopandas as gpd # type: ignore
import networkx as nx # type: ignore
import pandas as pd # type: ignore
import osmnx as ox # type: ignore
import numpy as np # type: ignore

from shapely.geometry import Polygon # type: ignore

## Func

In [13]:
# Загрузка gdf из OSM.
def download_gdf(id, tags, simplify=True):
    first_gdf = ox.geocode_to_gdf(f'R{id}', by_osmid=True)
    polygon_boundary = first_gdf.unary_union
    tags = {**tags}
    gdf = ox.features_from_polygon(polygon_boundary,
                                 tags=tags)
    gdf.reset_index(inplace=True)
    return gdf

## Main area

In [7]:
spb = ox.geocode_to_gdf('R337422', by_osmid=True) # Санкт-Петербург
lo_filter = "['highway'~'motorway|trunk|primary|secondary|tertiary|unclassified|residential|motorway_link|trunk_link|primary_link|secondary_link|tertiary_link']"
graph = ox.graph_from_polygon(spb.unary_union, network_type='drive', custom_filter=lo_filter)

In [28]:
spb

bbox_north  bbox_south  bbox_east  bbox_west   place_id  osm_type  \
0 0   60.244837   59.633783  30.759493  29.425758  181679261  relation   
  1   60.244837   59.633783  30.759493  29.425758  181679261  relation   

     osm_id        lat        lon     class            type  place_rank  \
0 0  337422  59.960674  30.158655  boundary  administrative           8   
  1  337422  59.960674  30.158655  boundary  administrative           8   

     importance addresstype              name  \
0 0    0.723213       state  Saint Petersburg   
  1    0.723213       state  Saint Petersburg   

                                          display_name  \
0 0  Saint Petersburg, Northwestern Federal Distric...   
  1  Saint Petersburg, Northwestern Federal Distric...   

                                              geometry  
0 0  POLYGON ((29.42576 60.19074, 29.42719 60.19039...  
  1  POLYGON ((30.06506 59.66830, 30.07331 59.66737...

In [9]:
n, e = ox.graph_to_gdfs(graph)

In [18]:
stop_bus = ox.features_from_polygon(spb.unary_union, tags={'highway': 'bus_stop'})
stop_bus = stop_bus[stop_bus.geometry.type != 'Polygon']

In [49]:
from h3 import h3

def lat_lng_to_h3(row):
    # Изменение порядка координат на (широта, долгота) для h3.polyfill
    coords = [(y, x) for x, y in row.geometry.exterior.coords]
    hexagons = list(h3.polyfill({'type': 'Polygon', 'coordinates': [coords]}, 8))
    return hexagons

def h3_to_geometry(h3_id):
    polygon = Polygon(h3.h3_to_geo_boundary(h3_id, geo_json=True))
    return polygon


# Применяем функцию к каждому полигону в GeoDataFrame
spb['hexagons'] = spb.apply(lat_lng_to_h3, axis=1)

# Создаем новый GeoDataFrame с индексом из гексагонов
gdf = gpd.GeoDataFrame(
    pd.Series(sum(spb.hexagons.tolist(), [])).apply(h3_to_geometry).to_frame(),
    geometry=0,
    crs='EPSG:4326'
)